# ML Dataset stats

I want to perform some verification on the ml dataset I generated.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import dask.distributed
import logging
import pathlib
import xarray as xr

from crims2s.dask import create_dask_cluster

In [ ]:
#ML_DATASET_DIR = '***BASEDIR***/mlready/2021-08-08-test/'
ML_DATASET_DIR = '***BASEDIR***/mlready/2021-08-08-test/'

## Boot dask cluster

In [ ]:
cluster = create_dask_cluster(cores=6)

In [ ]:
cluster

In [ ]:
cluster.scale(jobs=2)

In [ ]:
client = dask.distributed.Client(cluster)

In [ ]:
client

## Load dataset

In [ ]:
dataset_files = sorted([x for x in pathlib.Path(ML_DATASET_DIR).iterdir() if '0123.nc' in x.name])

In [ ]:
dataset_files[:10]

In [ ]:
def preprocess_one_example(dataset):
    return dataset.expand_dims('forecast_time')

In [ ]:
features = xr.open_mfdataset(dataset_files[:3], group='/features', concat_dim='forecast_time', preprocess=preprocess_one_example)

In [ ]:
model = xr.open_mfdataset(dataset_files[:3], group='/model', concat_dim='forecast_time', preprocess=preprocess_one_example)

In [ ]:
obs = xr.open_mfdataset(dataset_files[:3], group='/obs', concat_dim='forecast_time', preprocess=preprocess_one_example)

In [ ]:
model

In [ ]:
obs.isnull().sum().compute()

In [ ]:
features

In [ ]:
obs

In [ ]:
len(obs.lead_time)

In [ ]:
46 / 7

In [ ]:
sample

In [ ]:
sample = sample.persist()

In [ ]:
some_computed = sample.isnull().sum(dim=['latitude', 'longitude', 'realization'])

In [ ]:
some_computed.sum(dim=['variable', 'forecast_time']).compute().x.plot()

In [ ]:
some_computed.isel(lead_time=[0, 1]).sum(dim='variable').compute()

In [ ]:
sample.sel(variable='sst').isel(lead_time=slice(1, None)).sum(dim=['forecast_time', 'lead_time', 'realization']).compute().x.plot()

## Check y

In [ ]:
target = xr.open_mfdataset(dataset_files[:10], group='/target', concat_dim='forecast_time', preprocess=preprocess_one_example)

In [ ]:
target = target.compute()

In [ ]:
target.isnull().sum(dim=['category', 'forecast_time', 'lead_time', 'variable']).target.plot()

# Check obs

In [ ]:
obs = xr.open_mfdataset(dataset_files[:10], group='/obs', concat_dim='forecast_time', preprocess=preprocess_one_example)

In [ ]:
obs.isnull().sum(dim=['forecast_time', 'lead_time']).t2m.plot()